Lab Notebook (Draft) : Google Earth Engine as a Planetary-scale Data Repository
-------------------------------

The aim of the this lab notebook is an introduction to Google Earth Engine to collect planetary-scale data. Google Earth Engine combines a multi-petabyte catalog of satellite imagery and geospatial datasets (small selection of vector datasets) with planetary-scale analysis capabilities and makes it available for scientists, researchers, and developers to detect changes, map trends, and quantify differences on the Earth's surface. 

https://earthengine.google.com/

Below is the data catalog of whats available with Google Earth Engine.

https://developers.google.com/earth-engine/datasets/

 
It is important to note, an Earth Engine account is associated with a single Google account. To activate the earth engine account, you first need to get a Google Account and sign up with the link below. After signing up you can access GEE either through the javascript api and python api. 

https://signup.earthengine.google.com/#!/

Specifically, we will be using the Official Python Earthengine-api ***ee*** to get the data and ***geemap*** for mapping. [1] has a lot of resources,videos and guides in using geemap to visualise in Python. Run the following lines if you have not installed geemap.

```python
!pip install geemap
```
https://developers.google.com/earth-engine/guides/python_install

https://github.com/giswqs/geemap

*Reference*<br/>
[1] Wu, Q., (2020). geemap: A Python package for interactive mapping with Google Earth Engine. The Journal of Open Source Software, 5(51), 2305. https://doi.org/10.21105/joss.02305 <br/>

After you have installed geemap, you need to first authenticate and initialise Earth Engine once.

In [ ]:
# Authenticates and initializes Earth Engine
import ee #earthengine-api 
import geemap #geemap 


In [ ]:
ee.Authenticate()
ee.Initialize()

In [ ]:
import folium
import geopandas as gpd
import numpy as np
import pandas as pd

lets first visualise a google map using geemap that centres on London.

In [ ]:
Map=geemap.Map(center=[51.5,-0.14], zoom=14)

after that whenever you would like to open a map in the notebook, you just need to call it.

In [ ]:
Map

In this first lab notebook, we will focus on using Google Earth Engine to visualise satellite image data. 

First set a point using 
```python
ee.Geometry.Point.
```

In [ ]:
#this point is the same as the one used above - just more precise
point = ee.Geometry.Point([-0.1433,51.5058])

Let's first get a Landsat image (SGS Landsat 8 Surface Reflectance) centered on this point. 


### Landsat 

Landsat, a joint program of the USGS and NASA, has been observing the Earth continuously from 1972 through the present day. Today the Landsat satellites image the entire Earth's surface at a 30-meter resolution about once every two weeks, including multispectral and thermal data.

https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T1_SR

In [ ]:
# get landsat image
image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(point) \
    .filterDate('2016-01-01', '2016-12-31') \
    .sort('CLOUD_COVER') \
    .first() \
    .select('B[1-7]')\
    .divide(10000)

vis_params = {
    'min': 0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2']
}


Map.addLayer(image, vis_params, "Landsat-8")

In [ ]:
#Have another look at the map to see what you've done

#keep zooming out until you can see the extent of the image you have downloaded

Map

Let's now get Sentinel-2, a higher-resolution (10m pixel), multi-spectral image. 

### Sentinel2
The Copernicus Program is an ambitious initiative headed by the European Commission in partnership with the European Space Agency (ESA). The Sentinels are a constellation of satellites developed by ESA to operationalize the Copernicus program, which include all-weather radar images from Sentinel-1A and 1B, high-resolution optical images from Sentinel-2A and 2B, ocean and land data suitable for environmental and climate monitoring from Sentinel-3, as well as air quality data from Sentinel-5P.

https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR

let's first define a cloud masking func provided by google earth engine. This is to ensure that we map and analyse pictures of the land rather than clouds.

*Reference* <br/>
https://stackoverflow.com/questions/57841300/export-google-earth-engine-rgb-sentinel-2-imagery-to-google-drive-using-python-a

In [ ]:
# This is the cloud masking function provided by Google Earth Engine
def maskS2clouds(image):
    
    qa = image.select('QA60')
    #QA60 is the band (you can think of it like a variable) that stores information on cloud cover
    # the QA stands for quality
    
    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    # all the other bits are currently meaningless - awaiting higher dimensional input
    # for discussion on it see here https://gis.stackexchange.com/questions/307974/what-are-the-other-bits-in-sentinels-qa60-band

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0)
    mask = mask.bitwiseAnd(cirrusBitMask).eq(0)

    return image.updateMask(mask).divide(10000)



then get an image collection from sentinel2 that covers the point between 2018-0101 and 2020-01-30 with very little cloud.

```python
ee.ImageCollection
```

## Get Sentinal2 image collection

In [ ]:
# get sentinel2
imageCollect = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterBounds(point) \
    .filterDate('2017-01-01', '2018-1-30') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 1))\
    .map(maskS2clouds)\
    .sort('CLOUDY_PIXEL_PERCENTAGE', False) 

In [ ]:
# count 
count = imageCollect.limit(10).size() #you can set the limit on how many images you get from the collection
print('Count: ', str(count.getInfo())+'\n')

In [ ]:
# you can get properties of collection
imageCollect.propertyNames().getInfo()

In [ ]:
print (imageCollect.get('system:id').getInfo())

and then let's select the first image from the image collection.

```python
ee.Image
```

In [ ]:
image = ee.Image(imageCollect.sort('CLOUD_COVER').first())

similarly you can get the property of the image

In [ ]:
image.propertyNames().getInfo()


In [ ]:
print (image.get('system:index').getInfo())

In [ ]:
image.getMapId()

## visualise a single image

now, let's setup the vis params and then overlay a true-color satellite image [R,G,B] on the map. <br/>

As a sidenote, 
* you need to provide min and max values suitable for displaying reflectance from Earth Engine images. (min=0.0,max=0.30) 

* ('B4','B3','B2') standards for Red, Green and Blue Channel for the satellite images.

In [ ]:
vis_params = {
    'min': 0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2']
}

Map.centerObject(point, 14)
Map.addLayer(image, vis_params, "Sentinel-2")
Map.setCenter(-0.1433,51.5058, 12)

## visualise a collection of images based on region

In [ ]:
# get sentinel2
point = ee.Geometry.Point([-0.1433,51.5239])
geometry = point.buffer(2000)

imageMosaic = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterBounds(geometry) \
    .filterDate('2017-01-01', '2018-1-30') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 1))\
    .map(maskS2clouds)\
    .median()\
    .clip(geometry)

Map.addLayer(imageMosaic, vis_params, "Sentinel-2")

In [ ]:
#have a look at it!
Map

#because we used a point with buffer the new image on top is circular
# you can see the far higher level of detail by looking at the edge

## save image

now I want to save this image. you can use the EarthEngine-api. 

In [ ]:
image_rgb = imageMosaic.visualize(bands=['B4', 'B3', 'B2'],
                        min=[0.0, 0.0, 0.0],
                        max=[0.3, 0.3, 0.3]
                       )

# Assign export parameters.
task_config = {
    'region': geometry.coordinates().getInfo(), 
    'folder': 'Example_Folder_Name', # where you are saving
    'scale': 10, 
    'crs': 'EPSG:27700',
    'description': 'sentinal_London3' # name of the file
}
# Export Image
task = ee.batch.Export.image.toDrive(image_rgb, **task_config)
task.start()

or you can use geemap to save locally. 

In [ ]:
geemap.ee_export_image(image.select(['B4', 'B3', 'B2']), 
                       'temp_london.tif', 
                       scale=10, 
                       region=geometry,
                      file_per_band=False)

### Further Readings - Google Earth Engine Guide
For those that are interested in exploring further with Google Earth Engine. 
There are a lot of information in the official guide. 

https://developers.google.com/earth-engine/guides

In [ ]:
# For example, this gets the mean of the image
# https://developers.google.com/earth-engine/guides/reducers_reduce_region
image = image.reduce(ee.Reducer.mean())

pt = ee.Geometry.Point([-0.1433,51.5058])
geom = pt.buffer(800)

meanValue = image.reduceRegion(**{
  'reducer': ee.Reducer.mean(),
  'geometry': geom,
  'scale': 30, #scale of which to get the image. (30 is fine for )
  'maxPixels': 1e9
})

print (meanValue.getInfo())